In [1]:
!pip install joblib

In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import os
import seaborn as sns
import joblib
from sklearn.model_selection import train_test_split
import gzip
#import Recommender as Recommender
#import Evaluation as Evaluation

In [3]:
import dask as dd

In [4]:
def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

dfmeta = getDF('E:/Airtel Xstream/fyp datasets/meta_Electronics.json.gz')

In [5]:
dfmeta.head()

,asin,imUrl,description,categories,title,price,salesRank,related,brand
0,0132793040,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,0321732944,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,0439886341,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,0511189877,http://ecx.images-amazon.com/images/I/41HaAhbv...,The CLIKR-5 UR5U-8780L remote control is desig...,"[[Electronics, Accessories & Supplies, Audio &...",CLIKR-5 Time Warner Cable Remote Control UR5U-...,23.36,NaN,"{'also_viewed': ['B001KC08A4', 'B00KUL8O0W', '...",NaN
4,0528881469,http://ecx.images-amazon.com/images/I/51FnRkJq...,"Like its award-winning predecessor, the Intell...","[[Electronics, GPS & Navigation, Vehicle GPS, ...",Rand McNally 528881469 7-inch Intelliroute TND...,299.99,NaN,"{'also_viewed': ['B006ZOI9OY', 'B00C7FKT2A', '...",NaN


In [6]:
dfmeta.shape

(498196, 9)

In [7]:
dfreviews=pd.read_csv('E:/Airtel Xstream/fyp datasets/ratings_Electronics.csv', header=None,sep=',')

In [8]:
dfreviews.columns = ["reviewerID", "asin","overall","timestamp"]
dfreviews.head()

,reviewerID,asin,overall,timestamp
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200


In [9]:
dfreviews.shape

(7824482, 4)

In [33]:
product_ratings=pd.merge(dfreviews,dfmeta,on='asin',how='inner')

In [34]:
product_ratings.head()

,reviewerID,asin,overall,timestamp,imUrl,description,categories,title,price,salesRank,related,brand
0,AKM1MP6P0OYPR,0132793040,5.0,1365811200,http://ecx.images-amazon.com/images/I/31JIPhp%...,The Kelby Training DVD Mastering Blend Modes i...,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Mastering Blend Modes in A...,NaN,NaN,NaN,NaN
1,A2CX7LUOHB2NDG,0321732944,5.0,1341100800,http://ecx.images-amazon.com/images/I/31uogm6Y...,NaN,"[[Electronics, Computers & Accessories, Cables...",Kelby Training DVD: Adobe Photoshop CS5 Crash ...,NaN,NaN,NaN,NaN
2,A2NWSAGRHCP8N5,0439886341,1.0,1367193600,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
3,A2WNBOD3WNDNKT,0439886341,3.0,1374451200,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN
4,A1GI0U4ZRJA8WN,0439886341,1.0,1334707200,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger,"[[Electronics, Computers & Accessories, PDAs, ...",Digital Organizer and Messenger,8.15,{'Electronics': 144944},"{'also_viewed': ['0545016266', 'B009ECM8QY', '...",NaN


In [35]:
product_ratings.drop(['description','categories','price','salesRank','related','brand','timestamp'], axis=1, inplace=True)

In [36]:
product_ratings.head()

,reviewerID,asin,overall,imUrl,title
0,AKM1MP6P0OYPR,0132793040,5.0,http://ecx.images-amazon.com/images/I/31JIPhp%...,Kelby Training DVD: Mastering Blend Modes in A...
1,A2CX7LUOHB2NDG,0321732944,5.0,http://ecx.images-amazon.com/images/I/31uogm6Y...,Kelby Training DVD: Adobe Photoshop CS5 Crash ...
2,A2NWSAGRHCP8N5,0439886341,1.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger
3,A2WNBOD3WNDNKT,0439886341,3.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger
4,A1GI0U4ZRJA8WN,0439886341,1.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger


In [38]:
product_ratings = product_ratings.rename(columns={'reviewerID': 'userID', 'asin': 'prod_ID','title': 'prod_name',
                                                 'overall': 'rating' , 'imUrl':'imgurl'})
#product_ratings.to_csv('prod_ratings.csv')

In [39]:
product_ratings.head()

,userID,prod_ID,rating,imgurl,prod_name
0,AKM1MP6P0OYPR,0132793040,5.0,http://ecx.images-amazon.com/images/I/31JIPhp%...,Kelby Training DVD: Mastering Blend Modes in A...
1,A2CX7LUOHB2NDG,0321732944,5.0,http://ecx.images-amazon.com/images/I/31uogm6Y...,Kelby Training DVD: Adobe Photoshop CS5 Crash ...
2,A2NWSAGRHCP8N5,0439886341,1.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger
3,A2WNBOD3WNDNKT,0439886341,3.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger
4,A1GI0U4ZRJA8WN,0439886341,1.0,http://ecx.images-amazon.com/images/I/51k0qa8f...,Digital Organizer and Messenger


In [40]:
product_ratings.isnull().sum()

userID            0
prod_ID           0
rating            0
imgurl         5027
prod_name    293557
dtype: int64

In [47]:
df=product_ratings

In [48]:
df.dropna(inplace=True)

In [49]:
counts1=df['userID'].value_counts() 
counts=df['prod_ID'].value_counts()

In [50]:
df1=df[df['userID'].isin(counts1[counts1 >=200].index)]
df1.shape


(12030, 5)

In [51]:
df1=df1[df1['prod_ID'].isin(counts[counts >=200].index)]
df1.shape

(3567, 5)

In [52]:
df1.head()

,userID,prod_ID,rating,imgurl,prod_name
2162,A5JLAU2ARJ0BO,1400532655,1.0,http://ecx.images-amazon.com/images/I/51lBXZsM...,Barnes &amp; Noble Nook Color eReader
8147,A1T1YSCDW0PD25,B00000DM9W,5.0,http://ecx.images-amazon.com/images/I/41N1GD82...,Sony ICF-S79V Weather Band Shower Radio (Disco...
18168,A3CW0ZLUO5X2B1,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
18358,A22CW0ZHY3NJH8,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
19752,A231WM2Z2JL0U3,B00000K4KH,5.0,http://ecx.images-amazon.com/images/I/41SO%2B1...,Kensington 64068F MicroSaver Notebook Lock and...


In [53]:
ratingsd=df1.pivot(index='userID',columns= 'prod_ID',values='rating').fillna(0)
ratingsd.head()

prod_ID,1400532655,B00000DM9W,B00000K135,B00000K4KH,B00001P4XA,B00001P4ZH,B00001W0DI,B00001WRSJ,B00003CWDH,B00004SABB,...,B00HWMPRKM,B00HWMPRKW,B00HWSXVDG,B00IF70QCW,B00INNP5VU,B00ISGCAJM,B00IT1WJZQ,B00IVPU786,B00IX2VGFA,B00J46VVKE
userID,,,,,,,,,,,,,,,,,,,,,
A11KZ906QD08C5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A12DQZKRKTNF5E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A17BUUBOU0598B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1CMD08Z49PGKQ,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
A1EXGL6L0QQ0M5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0


In [54]:
ratingsd.shape

(45, 1955)

In [55]:
corrMatrix = ratingsd.corr(method='pearson')
corrMatrix.head(100)

prod_ID,1400532655,B00000DM9W,B00000K135,B00000K4KH,B00001P4XA,B00001P4ZH,B00001W0DI,B00001WRSJ,B00003CWDH,B00004SABB,...,B00HWMPRKM,B00HWMPRKW,B00HWSXVDG,B00IF70QCW,B00INNP5VU,B00ISGCAJM,B00IT1WJZQ,B00IVPU786,B00IX2VGFA,B00J46VVKE
prod_ID,,,,,,,,,,,,,,,,,,,,,
1400532655,1.000000,-0.022727,-0.032513,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
B00000DM9W,-0.022727,1.000000,-0.032513,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,0.333391,-0.022727
B00000K135,-0.032513,-0.032513,1.000000,-0.032513,-0.032513,-0.045061,-0.032513,-0.032513,-0.032513,-0.032513,...,-0.032513,-0.046214,-0.032513,-0.046512,-0.046214,0.374654,-0.032513,-0.032513,0.188467,-0.032513
B00000K4KH,-0.022727,-0.022727,-0.032513,1.000000,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
B00001P4XA,-0.022727,-0.022727,-0.032513,-0.022727,1.000000,-0.031499,-0.022727,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
B00009X6PH,1.000000,-0.022727,-0.032513,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
B00009XVCZ,-0.032513,-0.032513,-0.046512,-0.032513,-0.032513,-0.045061,-0.032513,-0.032513,-0.032513,-0.032513,...,-0.032513,-0.046214,-0.032513,-0.046512,-0.046214,-0.057639,-0.032513,-0.032513,-0.100003,-0.032513
B0000A0AJH,0.699025,-0.032513,-0.046512,-0.032513,-0.032513,-0.045061,-0.032513,-0.032513,-0.032513,0.699025,...,-0.032513,-0.046214,-0.032513,-0.046512,-0.046214,-0.057639,-0.032513,-0.032513,-0.100003,-0.032513


In [56]:
def get_similar(prod_id,rating):
    similar_ratings = corrMatrix[prod_id]*(rating-2.5)
    similar_ratings = similar_ratings.sort_values(ascending=False)
    #print(type(similar_ratings))
    return similar_ratings

In [57]:
def prod_name(prodid):
    return df1.loc[df1['prod_ID'] == prodid ].iloc[0]['prod_name']

In [58]:
def prod_img(prodid):
    return df1.loc[df1['prod_ID'] == prodid ].iloc[0]['imgurl']

In [59]:
items = [("1400532655",5),("B00000DM9W",3),("B00000K4KH",1),("B00001P4ZH",2)]
similar_products = pd.DataFrame()
for product,rating in items:
    similar_products = similar_products.append(get_similar(product,rating),ignore_index = True)

similar_products.head(10)

,1400532655,B00000DM9W,B00000K135,B00000K4KH,B00001P4XA,B00001P4ZH,B00001W0DI,B00001WRSJ,B00003CWDH,B00004SABB,...,B00HWMPRKM,B00HWMPRKW,B00HWSXVDG,B00IF70QCW,B00INNP5VU,B00ISGCAJM,B00IT1WJZQ,B00IVPU786,B00IX2VGFA,B00J46VVKE
0,2.500000,-0.056818,-0.081282,-0.056818,-0.056818,-0.078747,-0.056818,-0.056818,-0.056818,-0.056818,...,-0.056818,-0.080762,-0.056818,-0.081282,-0.080762,-0.100728,-0.056818,-0.056818,-0.174761,-0.056818
1,-0.011364,0.500000,-0.016256,-0.011364,-0.011364,-0.015749,-0.011364,-0.011364,-0.011364,-0.011364,...,-0.011364,-0.016152,-0.011364,-0.016256,-0.016152,-0.020146,-0.011364,-0.011364,0.166696,-0.011364
2,0.034091,0.034091,0.048769,-1.500000,0.034091,0.047248,0.034091,0.034091,0.034091,0.034091,...,0.034091,0.048457,0.034091,0.048769,0.048457,0.060437,0.034091,0.034091,0.104857,0.034091
3,0.015749,0.015749,0.022531,0.015749,0.015749,-0.500000,-0.250023,0.015749,-0.250023,0.015749,...,0.015749,0.022387,0.015749,0.022531,-0.257445,0.027921,0.015749,0.015749,0.048442,0.015749


In [60]:
final_results = pd.DataFrame(similar_products.sum().sort_values(ascending=False).head(20))
final_results.reset_index(inplace = True)
final_results.columns = ['prodid','rating']

In [64]:
def recommendations(items):
    similar_products = pd.DataFrame()
    for product,rating in items:
        similar_products = similar_products.append(get_similar(product,rating),ignore_index = True)

    final_results = pd.DataFrame(similar_products.sum().sort_values(ascending=False).head(20))
    final_results.reset_index(inplace = True)
    final_results.columns = ['prodid','rating']
    final_results['prodname'] = final_results['prodid'].apply(prod_name)
    final_results['imgurl'] = final_results['prodid'].apply(prod_img)
    
    return final_results

In [65]:
def original_product(items):
    original_products = pd.DataFrame()
    for product,rating in items:
           print(prod_name(product))

In [66]:
items = [("1400532655",1),("B00000DM9W",5),("B00000K4KH",5),("B00001P4ZH",5)]
original_product(items)
recommendations(items)

Barnes &amp; Noble Nook Color eReader
Sony ICF-S79V Weather Band Shower Radio (Discontinued by Manufacturer)
Kensington 64068F MicroSaver Notebook Lock and Security Cable (PC/Mac)
Koss PortaPro Headphones with Case


,prodid,rating,prodname,imgurl
0,B000089GN3,2.552909,Sennheiser PX 100 Lightweight Collapsible Head...,http://ecx.images-amazon.com/images/I/41BTCNXN...
1,B001NJ0D0Y,2.446700,Cooler Master Hyper N520 - CPU Cooler with Cop...,http://ecx.images-amazon.com/images/I/51AE940%...
2,B005GCSZD6,2.398525,ZyXEL PLA4205kit HomePlug AV 500 Mbps Powerlin...,http://ecx.images-amazon.com/images/I/31Tlcych...
3,B002TLT10I,2.398525,Sennheiser RS 170 Digital Wireless Headphone w...,http://ecx.images-amazon.com/images/I/41TEk6dY...
4,B009O7YUF6,2.398525,AMD FD8350FRHKBOX FX-8350 FX-Series 8-Core Bl...,http://ecx.images-amazon.com/images/I/41YGkpkY...
5,B0089IHBE8,2.398525,T-lock Incoming PRO Call Blocker with LCD Disp...,http://ecx.images-amazon.com/images/I/41a2wvpa...
6,B000IJV4BC,2.398525,Toshiba HD-A2 HD DVD Player,http://ecx.images-amazon.com/images/I/31Mpv9lH...
7,B000WYVBR0,2.398525,VideoSecu LED LCD TV Wall Mount for most 22&qu...,http://ecx.images-amazon.com/images/I/51Yw-Nch...
8,B0007QN18U,2.398525,Sennheiser RS 130 Wireless Surround Sound Head...,http://ecx.images-amazon.com/images/I/41tiDtlw...
9,B00224ZDFY,2.398525,Ortek Windows 7 Vista XP Media Center MCE PC R...,http://ecx.images-amazon.com/images/I/31sIeXVX...


In [67]:
items = [("1400532655",3),("B00000DM9W",1),("B00000K4KH",2),("B00001P4ZH",1)]
original_product(items)
recommendations(items)

Barnes &amp; Noble Nook Color eReader
Sony ICF-S79V Weather Band Shower Radio (Discontinued by Manufacturer)
Kensington 64068F MicroSaver Notebook Lock and Security Cable (PC/Mac)
Koss PortaPro Headphones with Case


,prodid,rating,prodname,imgurl
0,B0036Q7MV0,0.592703,Western Digital WD1002FAEX Caviar Black 1 TB S...,http://ecx.images-amazon.com/images/I/41y2cKTh...
1,B002BW6DBU,0.592703,Toshiba 640 GB USB 2.0 Portable External Hard ...,http://ecx.images-amazon.com/images/I/31l7k0Ox...
2,B000204SWE,0.592703,Philips DVP642 DivX-Certified Progressive-Scan...,http://ecx.images-amazon.com/images/I/31Y30KWT...
3,B00028DM96,0.592703,Sharp Electronics PW-E550 Electronic Dictionary,http://ecx.images-amazon.com/images/I/51MuF04e...
4,B00COYOAYW,0.592703,Nokia Lumia 521 (T-Mobile),http://ecx.images-amazon.com/images/I/41vJNspr...
5,B00007EDZG,0.592703,Nikon ML-L3 Wireless Remote Control,http://ecx.images-amazon.com/images/I/31%2BZC0...
6,B000DZMC3K,0.592703,Fujifilm Finepix V10 5.1MP Digital Camera with...,http://ecx.images-amazon.com/images/I/51xdtORu...
7,B000GBO9IM,0.592703,Sony 4 GB Memory Stick PRO DUO Media (MSX-M4GS...,http://ecx.images-amazon.com/images/I/51yk7LFC...
8,1400532655,0.592703,Barnes &amp; Noble Nook Color eReader,http://ecx.images-amazon.com/images/I/51lBXZsM...
9,B00009X6PH,0.592703,Cisco-Linksys WUSB54G Wireless-G USB Adapter,http://ecx.images-amazon.com/images/I/41E1V9TZ...


In [126]:
newdf = pd.read_csv('E:/Airtel Xstream/FYP heroku/Building UI For ML Apps - Par1/web2py/applications/movies/static/item_similarity_df.csv')

In [68]:
newdf.head()

NameError: name 'newdf' is not defined

In [128]:
corrMatrix.to_csv('prod_similarity.csv')

In [129]:
corrdf = pd.read_csv('prod_similarity.csv')

In [132]:
corrdf.head() 

,prod_ID,1400532655,B00000DM9W,B00000K135,B00000K4KH,B00001P4XA,B00001P4ZH,B00001W0DI,B00001WRSJ,B00003CWDH,...,B00HWMPRKM,B00HWMPRKW,B00HWSXVDG,B00IF70QCW,B00INNP5VU,B00ISGCAJM,B00IT1WJZQ,B00IVPU786,B00IX2VGFA,B00J46VVKE
0,1400532655,1.000000,-0.022727,-0.032513,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
1,B00000DM9W,-0.022727,1.000000,-0.032513,-0.022727,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,0.333391,-0.022727
2,B00000K135,-0.032513,-0.032513,1.000000,-0.032513,-0.032513,-0.045061,-0.032513,-0.032513,-0.032513,...,-0.032513,-0.046214,-0.032513,-0.046512,-0.046214,0.374654,-0.032513,-0.032513,0.188467,-0.032513
3,B00000K4KH,-0.022727,-0.022727,-0.032513,1.000000,-0.022727,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727
4,B00001P4XA,-0.022727,-0.022727,-0.032513,-0.022727,1.000000,-0.031499,-0.022727,-0.022727,-0.022727,...,-0.022727,-0.032305,-0.022727,-0.032513,-0.032305,-0.040291,-0.022727,-0.022727,-0.069905,-0.022727


In [69]:
df1.to_csv('prod_ratings.csv',index=False)

In [70]:
proddsdf = pd.read_csv('prod_ratings.csv')

In [71]:
proddsdf.head()

,userID,prod_ID,rating,imgurl,prod_name
0,A5JLAU2ARJ0BO,1400532655,1.0,http://ecx.images-amazon.com/images/I/51lBXZsM...,Barnes &amp; Noble Nook Color eReader
1,A1T1YSCDW0PD25,B00000DM9W,5.0,http://ecx.images-amazon.com/images/I/41N1GD82...,Sony ICF-S79V Weather Band Shower Radio (Disco...
2,A3CW0ZLUO5X2B1,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
3,A22CW0ZHY3NJH8,B00000K135,5.0,http://ecx.images-amazon.com/images/I/71QD0N8R...,Brother M231 1/2-Inch Black on White Tape for ...
4,A231WM2Z2JL0U3,B00000K4KH,5.0,http://ecx.images-amazon.com/images/I/41SO%2B1...,Kensington 64068F MicroSaver Notebook Lock and...
